## 必要ライブラリのインストール

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#ja.zip was downloaded from https://drive.google.com/file/d/0B0ZXk88koS2KMzRjbnE4ZHJmcWM/view  via google drive
!unzip  /content/drive/My\ Drive/ja.zip

In [33]:
import MeCab
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## ベクトル化関数

In [91]:
def tokenizer(text):
    tokens=[]
    tagger = MeCab.Tagger("-Ochasen")
    node = tagger.parse(text).split('\n')
    for n in node:
        if ( '名詞' in n):
            tokens.append(n.split('\t')[0])
        if(n=='EOS'):
            break
    return tokens

In [ ]:
# wikipedia pretrained model
!pip install gensim
import gensim
model = gensim.models.Word2Vec.load('ja.bin')

In [75]:
def personal_vector(words):
    tokens=tokenizer(words)
    num_token=0
    v=[0.]*300
    for t in tokens:
        try :
            v=v+model[t]
            num_token+=1
        except KeyError:
            pass
        return v/num_token

In [ ]:
personal_vector('山と川')

In [94]:
def hotel_vector(hotel_id):
    v=[0.]*300
    num_token=0
    for i in range(0,100,20):#100件取得
        load_url = "https://review.travel.rakuten.co.jp/hotel/voice/"+str(hotel_id)+"/?f_sort=4&f_next="+str(i)
        html = requests.get(load_url)
        soup = BeautifulSoup(html.content, "html.parser")
        elements = soup.find_all(class_='commentSentence')
        for e in elements:
            if e is not []:
                ts=tokenizer(e.text)
            for t in ts:
                try:
                    v+=model[t]
                    num_token+=1
                except KeyError:
                    pass
    v=v/num_token 
    return v

In [95]:
hoshino_resort_sapporo=hotel_vector(165109)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [96]:
model.similar_by_vector(hoshino_resort_sapporo)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ご', 0.5899515151977539),
 ('レストラン', 0.5697121024131775),
 ('ショッピング', 0.5278029441833496),
 ('料亭', 0.5019051432609558),
 ('メニュー', 0.5013384819030762),
 ('カフェ', 0.49937140941619873),
 ('スイーツ', 0.49858057498931885),
 ('グルメ', 0.4977513551712036),
 ('スナック', 0.49663758277893066),
 ('ファミリーレストラン', 0.4934089183807373)]

## 適当にどんなワードが多いか見てみる

In [133]:
dictionary=set()
num_token=0
for i in range(0,20,20):#20件取得
    load_url = "https://review.travel.rakuten.co.jp/hotel/voice/"+str(165109)+"/?f_sort=4&f_next="+str(i)
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, "html.parser")
    elements = soup.find_all(class_='commentSentence')
    for e in elements:
        if e is not []:
            ts=tokenizer(e.text)
        for t in ts:
            try:
                tmp=model[t]
                num_token+=1
                dictionary.update([t])
            except KeyError:
                pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [134]:
counter={}
for d in dictionary:
    counter[d]=0

In [135]:
for i in range(0,20,20):#20件取得
    load_url = "https://review.travel.rakuten.co.jp/hotel/voice/"+str(165109)+"/?f_sort=4&f_next="+str(i)
    html = requests.get(load_url)
    soup = BeautifulSoup(html.content, "html.parser")
    elements = soup.find_all(class_='commentSentence')
    for e in elements:
        if e is not []:
            ts=tokenizer(e.text)
        for t in ts:
            try:
                tmp=model[t]
                counter[t]+=1
            except KeyError:
                pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
for k, v in sorted(counter.items(), key=lambda x: -x[1]):
    print(str(k) + ": " + str(v))